# Model Benchmarking using the STM32Cube.AI Developer Cloud Service

In [12]:
####################### Please set your model and target board to benchmark. #######################
target_board = 'STM32F769I-DISCO'                                   # you can check available cloud boards in the paragraph '2. Check the availability of the MCU board' mentioned below.
benchmark_model_name_list = ['mcunet-512kb-2mb_imagenet.tflite']    # model file location: ./.models/{benchmark_model_name}

---

In [15]:
import getpass
import os

from stm32_api.file import FileManager
from stm32_api.benchmark import BenchmarkManager

from stm32_api.utils import CliParameters
from stm32_api.analyze import analyze_footprints, analyze_inference_time, get_inference_time

In [2]:
result_dl_dir = "./.benchmark/"
benchmark_model_dir = "./.models/"

---

## 1. Login to the STM32Cube.AI Developer Cloud Service

In [3]:
stm32_email = input("Enter your STM32 Account: ")

In [ ]:
print("Enter your STM32 Password: ")
stm32_password = getpass.getpass()

In [5]:
try:
    file_manager = FileManager(stm32_email, stm32_password)
    print('STM32Cube.AI Dev Cloud Login success')
except Exception as e:
    print("Error: please verify your credentials")

---

## 2. Check the availability of the MCU board in the STM32Cube.AI Developer Cloud Service

In [7]:
benchmark_manager = BenchmarkManager(
    net = None,
    file_manager = file_manager,
    path = result_dl_dir,
    stm_version = "1.0.0",
)

In [ ]:
boards = benchmark_manager.get_benchmark_boards()
board_names = [boards[i].name for i in range(len(boards))]
print("Available boards:", board_names)

assert target_board in board_names, f'target MCU board {board_name} is not available now'

---

## 3. Upload the model and perform the benchmarking

In [13]:
timeout = 1200
analysis = False

benchmark_options = {               # CliParameters (stm32_api/utils/types.py)
    "optimization": "balanced",
    "allocateInputs": True,
    "allocateOutputs": True,
    "noOnnxIoTranspose": False,
    "fromModel": "user",
}

In [ ]:
assert isinstance(benchmark_model_name_list, list), 'benchmark_model_name_list should be a list'

latency_dict = {}

for model_name in benchmark_model_name_list:
    model_file_path = os.path.join(benchmark_model_dir, model_name)

    if not os.path.isfile(model_file_path):
        print(f"Please check the model file exists: {model_file_path}")
        continue

    ################## 1. Upload the model to the Cloud ##################
    print(f"Uploading the '{model_name}' model file to the STM32 Cloud")
    file_manager.upload_model(file_path=model_file_path, model_name=model_name)
    benchmark_manager.model_name = model_name


    ################## 2. Benchmark the model on the Cloud ##################
    print(f'Start benchmarking the model: {model_name}')
    try:
        result = benchmark_manager.benchmark(
            CliParameters(model=model_name, **benchmark_options), 
            target_board,
            timeout=timeout,
            analysis=analysis,
        )
        if analysis:
            latency_dict[model_name] = get_inference_time(report=result)
    except Exception as e:
        print(f"Error: {e}")


    ################## 3. Download the benchmark result on the Cloud ##################
    benchmark_manager.get_result_given_id(benchmark_manager.bid_list[-1])


    ################## 4. Delete the result and file on the Cloud ##################
    benchmark_manager.delete_result_given_id(benchmark_manager.bid_list[-1])
    file_manager.delete_model(model_name)

if analysis:
    print(f"latency_dict: {latency_dict}")